In [1]:
import pandas as pd
from datasets import Dataset, load_dataset
from tqdm import tqdm

In [2]:
tqdm.pandas()

## Load your dataset

In [3]:
# train or test part
PART = 'test'

In [4]:
dss = load_dataset('EvaKlimentova/knots_AF')
dss = dss[PART]
dss = dss.filter(lambda x: x['label']==0)
df = pd.DataFrame(dss)
print(df.shape)

Using custom data configuration EvaKlimentova--knots_AF-cd42605bc53e4fb3
Found cached dataset parquet (/home/jovyan/.cache/huggingface/datasets/EvaKlimentova___parquet/EvaKlimentova--knots_AF-cd42605bc53e4fb3/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)


  0%|          | 0/2 [00:00<?, ?it/s]

(39758, 14)


## Download structure from AF database based on ID and run topoly

In [5]:
from topoly import alexander

In [6]:
import requests

def get_AF_knot(ID):
    response = requests.get(f'https://alphafold.ebi.ac.uk/files/AF-{ID}-F1-model_v4.pdb')
    if not response.ok or response.text == 'N/A':
        return {}
    structure = response.text
    with open('tmp.pdb', 'w') as f:
        f.write(structure)
    try:
        knot_types = alexander('tmp.pdb')
    except:
        return {}
    return knot_types

In [ ]:
df['AF_knot_prediction'] = df['ID'].progress_apply(get_AF_knot)

  7%|▋         | 2944/39758 [2:41:55<53:33:25,  5.24s/it]

In [ ]:
df['AF_knotted'] = df['AF_knot_prediction'].apply(lambda x: 0 if ('0_1' in x.keys() and x['0_1'] > 0.5) else 1)

In the final csv, 'AF_knotted' column represents summarized topoly result: 0 -> quite surely unknotted, 1 -> might be knotted

When we get empty result in the 'AF_knot_prediction' column, it means some error occured during the computation (eg. protein structure couldn't be downloaded from AF database, the downloaded structure is somehow corrupted, topoly throwed some error...)


In [ ]:
df

In [ ]:
df.to_csv('AF_recomputed_' + PART + '.csv', index=False)